### Run GLM for power analysis

In [1]:
%load_ext autoreload

In [19]:
%autoreload 2


In [3]:
import scanpy as sc
import seaborn as sns
import pandas as pd
import numpy as np
import itertools
import logging
import statsmodels.api as sm

import pymare as meta

In [4]:
import sys
sys.path.append('/home/ssm-user/Github/memento')

In [5]:
data_path = '/data_volume/memento/'

In [6]:
logging.basicConfig(
    format="%(asctime)s %(process)-7s %(levelname)-8s %(message)s",
    level=logging.INFO, 
    datefmt="%Y-%m-%d %H:%M:%S",
)
logging.captureWarnings(True)

In [7]:
import memento.model.rna as rna

In [8]:
pop = 'asian'
inds = 50
rep=0
ct = 'cM'

In [9]:
data_path  = '/data_volume/memento/lupus/'

pos = pd.read_csv(data_path + 'mateqtl_input/sampled_pos/{}_{}_{}.tsv'.format(pop, inds ,rep), sep='\t', index_col=0)
cov = pd.read_csv(data_path + 'mateqtl_input/sampled_cov/{}_{}_{}.tsv'.format(pop, inds, rep), sep='\t', index_col=0).T

# Randomly generate genotypes
shuffled_pos = pos.copy()
shuffled_pos[shuffled_pos.columns] = np.random.choice([0, 1, 2], size=pos.shape)
onek_replication = pd.read_csv(data_path + 'filtered_onek_eqtls.csv')

adata = sc.read(data_path + 'single_cell/{}_{}.h5ad'.format(pop, ct))
adata = adata[adata.obs.ind_cov.isin(pos.columns)].copy()

In [10]:
adata.obs['q'] = 0.1
rna.MementoRNA.setup_anndata(
    adata=adata,
    q_column='q',
    label_columns=['ind_cov'])

2023-07-14 11:10:28 2423080 INFO     setup_anndata: creating groups
2023-07-14 11:10:28 2423080 INFO     setup_anndata: computing cell sizes


In [11]:
cov_df = cov.loc[[x.split('^')[1] for x in adata.uns['memento']['groups']]]
donor_df = pos[[x.split('^')[1] for x in adata.uns['memento']['groups']]].T
shuffled_donor_df =  shuffled_pos[[x.split('^')[1] for x in adata.uns['memento']['groups']]].T

cov_df.index = 'memento_group^'+cov_df.index
cov_df = sm.add_constant(cov_df)
donor_df.index = 'memento_group^'+donor_df.index
shuffled_donor_df.index = 'memento_group^'+shuffled_donor_df.index

In [12]:
gene_snp_pairs = onek_replication.query('cg_cov == "{}"'.format(ct))
gene_to_snp = dict(gene_snp_pairs[gene_snp_pairs.gene.isin(adata.var.index)].groupby('gene').rsid.apply(list))

In [13]:
# Define expressed genes
adata.var['expr_genes'] = (adata.X.mean(axis=0).A1 > 0.01)
adata = adata[:, adata.var['expr_genes']]

In [14]:
model = rna.MementoRNA(adata=adata)
model.compute_estimate(
estimand='mean',
get_se=True,
n_jobs=30,
)

2023-07-14 11:10:46 2423080 INFO     compute_estimate: running estimators for ['sum', 'mean', 'log_mean', 'log1p_mean', 'se_sum', 'se_mean', 'se_log_mean', 'se_log1p_mean', 'total_umi', 'cell_count']
2023-07-14 11:10:46 2423080 INFO     compute_estimate: gene_list is None, using all genes in AnnData object
2023-07-14 11:10:46 2423080 INFO     compute_estimate: getting estimates for memento_group^HC-519 using 30 parallel jobs
2023-07-14 11:10:46 2423080 INFO     compute_estimate: getting estimates for memento_group^1791_1791 using 30 parallel jobs
2023-07-14 11:10:46 2423080 INFO     compute_estimate: getting estimates for memento_group^1240_1240 using 30 parallel jobs
2023-07-14 11:10:46 2423080 INFO     compute_estimate: getting estimates for memento_group^HC-022 using 30 parallel jobs
2023-07-14 11:10:46 2423080 INFO     compute_estimate: getting estimates for memento_group^1771_1771 using 30 parallel jobs
2023-07-14 11:10:46 2423080 INFO     compute_estimate: getting estimates for m

In [69]:
result = model.differential_mean(covariates=covariate, treatments=treatment, family='quasiGLM', n_jobs=30, verbose=2)

In [26]:
sm.families.links.log()

In [35]:
result

,treatment,coef,se,pval
gene,,,,
RP11-108M9.4,1:16796721,-0.024074,0.394824,0.951380
PADI2,1:17414305,0.267382,0.492513,0.587203
CDA,1:20935986,0.242343,0.459535,0.597940
CDC42,1:22414785,0.044723,0.412463,0.913655
PNRC2,1:24375663,-0.389440,0.409567,0.341676
...,...,...,...,...
AP001055.6,21:45636583,-0.113024,0.303645,0.709726
ITGB2,21:46328099,-0.493111,0.532804,0.354705
ITGB2-AS1,21:46349496,0.082331,0.391374,0.833383


In [179]:
def quasi_nb_var(mean, scale, dispersion):
    
    return scale*(mean + dispersion*mean**2)


def quasi_nb_objective(scale, dispersion, mean, variance):
    
    valid = (mean > 0) & (variance > 0)
    pred_y = np.log(quasi_nb_var(mean[valid], scale, dispersion))
    y = np.log(variance[valid])
    
    return ((pred_y-y)**2).mean()

    
    

def fit_quasi_nb(mean, variance):

    dispersion0 = 1
    scale0 = np.median((mean[variance > 0]/variance[variance > 0]))
        
    optim_obj = lambda params: quasi_nb_objective(params[0], params[1], mean, variance)

    res =  minimize(
        optim_obj, 
        [scale0, dispersion0],
        bounds=[(1e-5,None), (1e-10, 10)],
        method='Nelder-Mead'
    )
    return res.x

In [168]:
n_groups = model.estimates['mean'].shape[0]

In [169]:
group_idx = 0

In [170]:
mean = expr.iloc[group_idx].values
variance = sampling_variance.iloc[group_idx].values

In [171]:
optim_obj = lambda params: quasi_nb_objective(params[0], params[1], mean, variance)

In [172]:
optim_obj([1.6, 0.1])

5.525925

In [178]:
minimize(
        optim_obj, 
        [1.6, 1],
        bounds=[(1e-5,None), (1e-10, 10)],
        method='Nelder-Mead',
    )

       message: Optimization terminated successfully.
       success: True
        status: 0
           fun: 0.025494040921330452
             x: [ 6.138e-01  1.000e-10]
           nit: 74
          nfev: 138
 final_simplex: (array([[ 6.138e-01,  1.000e-10],
                       [ 6.137e-01,  1.000e-10],
                       [ 6.137e-01,  1.000e-10]]), array([ 2.549e-02,  2.549e-02,  2.549e-02]))

In [122]:

fit_quasi_nb(    
        mean = mean,
        variance = variance)

array([0.47579199, 0.        ])

In [92]:
quasi_nb_objective(1.6, 0.1, mean, variance)

5.525925

In [180]:
expr = (
    model.estimates['mean']/
    model.adata.uns['memento']['umi_depth']*
    model.estimates['total_umi'].values)
count_multiplier = model.estimates['total_umi'].values/model.adata.uns['memento']['umi_depth']
sampling_variance =  (model.estimates['se_mean']**2)*count_multiplier**2

# Fit within-sample variance function parameters
intra_var_scale = np.zeros(n_groups)
intra_var_dispersion = np.zeros(n_groups)
for group_idx in range(n_groups):
    intra_var_scale[group_idx], intra_var_dispersion[group_idx] = fit_quasi_nb(    
        mean = expr.iloc[group_idx].values,
        variance = sampling_variance.iloc[group_idx].values)

In [184]:
result = model.differential_mean(
    covariates=cov_df, 
    treatments=donor_df,
    treatment_for_gene=gene_to_snp,
    family='quasiGLM',
    verbose=2,
    n_jobs=1)

shuffled_result = model.differential_mean(
    covariates=cov_df, 
    treatments=shuffled_donor_df,
    treatment_for_gene=gene_to_snp,
    family='quasiGLM',
    verbose=2,
    n_jobs=5)

rr = result.query('pval<0.05').shape[0]/result.shape[0]
fpr =shuffled_result.query('pval<0.05').shape[0]/shuffled_result.shape[0]
print(f'replication rate: {rr}')
print(f'FPR rate: {fpr}')

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
2023-07-14 11:43:25 2423080 WARNING  /home/ssm-user/anaconda3/envs/single_cell/lib/python3.10/site-packages/statsmodels/genmod/families/links.py:516: RuntimeWarning: overflow encountered in exp
  return np.exp(z)

2023-07-14 11:43:25 2423080 WARNING  /home/ssm-user/anaconda3/envs/single_cell/lib/python3.10/site-packages/statsmodels/genmod/families/family.py:132: RuntimeWarning: invalid value encountered in multiply
  return 1. / (self.link.deriv(mu)**2 * self.variance(mu))

2023-07-14 11:43:25 2423080 WARNING  /home/ssm-user/anaconda3/envs/single_cell/lib/python3.10/site-packages/statsmodels/genmod/generalized_linear_model.py:1211: RuntimeWarning: invalid value encountered in multiply
  wlsendog = (lin_pred + self.family.link.deriv(mu) * (self.endog-mu)

2023-07-14 11:43:25 2423080 WARNING  fit_loglinear: HLA-DRB5, 6:32599999 f

replication rate: 0.3768996960486322
FPR rate: 0.0182370820668693


In [54]:
gene ='PADI2'
t = '1:17414305'
design_matrix = pd.concat([cov_df, donor_df[[t]]], axis=1)

In [18]:
groups_in_test = cov_df.index.tolist()
test_estimates = {est:res.loc[groups_in_test] for est,res in model.estimates.items()}

test_estimates['log1p_mean'] = np.log(test_estimates['mean']+1)
l = np.log((test_estimates['mean']+1) - test_estimates['sem'])
u = np.log((test_estimates['mean']+1) + test_estimates['sem'])
test_estimates['log1p_sem'] = (u-l)/2

expr = test_estimates['log1p_mean']
expr_sem = test_estimates['log1p_sem']
min_sem = expr_sem[expr_sem > 0].min(axis=0)
for col in expr_sem.columns:
    expr_sem[col] = expr_sem[col].replace(0.0, min_sem[col])

In [19]:
y=expr.loc[:,[gene]]
X=design_matrix.values
v=expr_sem.loc[:,[gene]]**2

In [20]:
# MetaRegression
data = meta.core.Dataset(
    y=y,
    X=X,
    v=v,
    X_names=design_matrix.columns)

dsl = meta.estimators.WeightedLeastSquares()
dsl.fit_dataset(data)
result = dsl.summary()
coef = float(result.get_fe_stats()['est'][-1])
p = float(result.get_fe_stats()['p'][-1])

print(result.to_df().iloc[-1, :])
print(result.get_heterogeneity_stats())
print(result.get_re_stats())

name        (1:17414305,)
estimate         0.007764
se               0.001547
z-score          5.017171
p-value          0.000001
ci_0.025         0.004731
ci_0.975         0.010797
Name: 35, dtype: object
{'Q': array([48.41112721]), 'p(Q)': array([1.12369722e-05]), 'I^2': array([71.08102867]), 'H': array([1.85955308])}
{'tau^2': 0.0, 'ci_l': array([1.05065384e-05]), 'ci_u': array([0.00010914])}


In [21]:
ols_model= sm.OLS(y, design_matrix).fit()
print(ols_model.summary())

                            OLS Regression Results                            
Dep. Variable:                  PADI2   R-squared:                       0.873
Model:                            OLS   Adj. R-squared:                  0.587
Method:                 Least Squares   F-statistic:                     3.046
Date:                Fri, 30 Jun 2023   Prob (F-statistic):             0.0123
Time:                        19:35:07   Log-Likelihood:                 208.26
No. Observations:                  50   AIC:                            -346.5
Df Residuals:                      15   BIC:                            -279.6
Df Model:                          34                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const              0.0149      0.016      0.

In [ ]:
np.linalg.matrix_rank(design_matrix)

36

In [ ]:
design_matrix

,age,Female,status,PC1_e,PC2_e,PC3_e,PC4_e,PC5_e,PC6_e,PC7_e,...,batch_cov_b_15,batch_cov_b_2,batch_cov_b_3,batch_cov_b_4,batch_cov_b_5,batch_cov_b_6,batch_cov_b_7,batch_cov_b_8,batch_cov_b_9,1:17414305
memento_group^1791_1791,66.0,0.0,0.0,-7.161831,10.069257,4.624538,-22.322425,5.751324,0.806356,3.677423,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1
memento_group^1240_1240,23.0,0.0,0.0,-8.648941,13.411016,5.855519,7.568020,-8.009542,-0.186709,-1.124865,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0
memento_group^HC-022,74.0,0.0,1.0,2.728118,-13.795299,1.074720,7.365269,10.467058,-6.088037,-5.948432,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
memento_group^1771_1771,49.0,0.0,0.0,-7.359370,10.981154,-0.085993,-23.435739,-0.616778,4.374860,2.659662,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1
memento_group^1334_1334,52.0,0.0,0.0,5.260662,-28.224529,3.845200,-4.865216,-2.921565,-2.306062,-3.568229,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
memento_group^HC-573,31.0,0.0,1.0,8.553464,-22.598488,18.165417,-5.401827,-2.331321,-0.081234,-10.778825,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
memento_group^1754_1754,45.0,0.0,0.0,-8.188705,12.238516,1.101978,-22.625022,-0.472781,2.883861,3.796883,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0
memento_group^HC-571,52.0,0.0,1.0,13.338191,-28.923302,20.733674,4.683934,-11.270210,-7.643216,-5.951474,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
memento_group^1886_1886,23.0,0.0,0.0,-0.590112,-8.827822,-5.990130,9.029696,11.168228,-1.493827,-7.803633,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
memento_group^1022_1022,56.0,0.0,0.0,-3.538297,11.315355,21.996052,10.318588,-13.082991,-1.902776,-3.783664,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [54]:
result.query('coef == 0')

,treatment,coef,pval
gene,,,
PADI2,1:17414305,0.0,1.0
PNRC2,1:24375663,0.0,1.0
FHL3,1:38465315,0.0,1.0
ITGB3BP,1:63893394,0.0,1.0
GSTM4,1:110172362,0.0,1.0
...,...,...,...
UPK3A,22:45673477,0.0,1.0
OLIG1,21:34432792,0.0,1.0
AP001053.11,21:45649640,0.0,1.0


In [55]:
result.query('pval <0.05')

,treatment,coef,pval
gene,,,
CDA,1:20935986,0.105587,2.442491e-14
CD52,1:26645806,0.290659,2.026619e-06
C1orf122,1:38235343,-0.011924,4.227407e-03
NDUFS5,1:39465139,-0.077633,3.822534e-03
PPT1,1:40559207,0.102415,2.993161e-12
...,...,...,...
MAP3K7CL,21:30533397,0.040103,6.061818e-14
IFNGR2,21:34776300,0.035621,2.710907e-04
HMGN1,21:40731931,0.033734,1.507120e-02


In [22]:
model.estimates['log1p_mean'] = np.log(model.estimates['mean']+1)
l = np.log((model.estimates['mean']+1) - model.estimates['sem'])
u = np.log((model.estimates['mean']+1) + model.estimates['sem'])
model.estimates['log1p_sem'] = (u-l)/2

In [23]:
dd

In [36]:
result = model.differential_mean(
    covariates=cov_df, 
    treatments=donor_df,
    treatment_for_gene=gene_to_snp,
    family='NB',
    verbose=2,
    n_jobs=5)

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  74 tasks      | elapsed:    1.0s
/home/ssm-user/anaconda3/envs/single_cell/lib/python3.10/site-packages/statsmodels/genmod/families/links.py:516: RuntimeWarning: overflow encountered in exp
  return np.exp(z)
/home/ssm-user/anaconda3/envs/single_cell/lib/python3.10/site-packages/statsmodels/genmod/families/family.py:1342: RuntimeWarning: divide by zero encountered in divide
  endog_mu = self._clean(endog / mu)
/home/ssm-user/anaconda3/envs/single_cell/lib/python3.10/site-packages/statsmodels/genmod/families/family.py:1346: RuntimeWarning: divide by zero encountered in log
  resid_dev -= endog_alpha * np.log(endog_alpha / mu_alpha)
/home/ssm-user/anaconda3/envs/single_cell/lib/python3.10/site-packages/statsmodels/genmod/families/varfuncs.py:265: RuntimeWarning: overflow encountered in square
  return p + self.alpha*p**2
/home/ssm-user/anaconda3/envs/single_cell/lib/python3.10/site-pack

In [37]:
result.query('pval < 0.05')

,treatment,coef,pval
gene,,,
CHST13,3:126268953,0.691833,6.108333e-04
RP11-539L10.3,4:6675421,-0.665931,1.192296e-08
SHROOM1,5:132613418,0.344046,2.402571e-06
HLA-DQA1,6:32606756,-1.525906,4.212932e-02
CCZ1B,7:6855646,-1.682542,3.673926e-03
TMEM176A,7:150477909,-2.123900,4.185953e-02
HP,16:72118324,-1.849975,7.743754e-04
GSTT1,22:24334948,-2.280182,2.985696e-120
